# Dijkstra algorithm for sparse graphs

Implement the refined Dijkstra algorithm from the paper [An improved Dijkstra’s shortest path algorithm for sparse network](https://www.sciencedirect.com/science/article/abs/pii/S0096300306008563). Check if its performance is faster than the simple heapq Dijkstra 

The core change is in the handling of the list of $l(\cdot)$'s of tentative distances. Instead of using a heap data structure and the likes: for the extracting of the minimum distance, insertion/updates of tentative distances, and maintaining the sorted order of the list, we make use of a different method for the insertion of the new distance $l(\cdot)$ and where to insert it into the list (Steps $2.1.1$ and $2.1.2$)

- $2.1.1)$ Let $high_i^v$ be the position of entry $l(v)$ at the list $l$
- $2.1.2)$ Sort list $l$ by reinserting the entry $l(v)$ in a proper position between position $low_i$ and $high_i^v$ of list $l$

So for each iteration it just takes the next position from the list as the minimum: since the sorting of the list is handled by the previous two process of properly reinserting the entry $l(v)$ and upholding the order of the list

Conceptually, $2.1.2$ is just a binary search between the length of $low_i$ and $high_i^v$ since the sublist across this length is nondecreasing. Traditionally binary search divides the length to find the midpoint, but here we incorporate a step size vector which gives the midpoint for each length (avoiding division) in the binary search

In [ ]:
# Heap implementation
import heapq


def dijkstra_heap(G, source, target):
    visited = set()
    queue = [(0, source, None)]  # (tentative cost, vertex, parent)
    parents = {}
    
    while queue:
        vertex_cost, vertex, parent = heapq.heappop(queue)  # Get lowest cost vertex from the queue

        if vertex in visited:
            continue

        visited.add(vertex) # Consider vertex as visited
        parents[vertex] = parent
        
        # End algorithm when vertex visited is the goal node
        if vertex == target:
            break

        # Relax neighbor vertices of vertex
        for _, neighbor, edge_length in G.out_edges(vertex, data="length"):
            if neighbor in visited:
                continue

            # No need to check if new cost is an improvement, just add it to the queue; other costs will be disregarded later on
            neighbor_cost = vertex_cost + edge_length 
            heapq.heappush(queue, (neighbor_cost, neighbor, vertex))

    return parents

The structure of the code, I think, will still closely follow the heap implemenetation previously. But the popping and inserting of vertices will be totally different (and also the data structures used). I think it would be beneficial to also try to retain the efficiency quirks of the heap implementation: not adding all the vertices from the start; multiple costs for same vertices existing in the queue. But not sure if they would apply

One issue I'm seeing is that we'd have to find a way of getting the position of the vertex in the list. Suppose that we successfully implement the data structure replacing the heap process; but the introduction of $O(n)$ searches might make the performance all the same if not worse

In [ ]:
# Sparse implementation

def dijkstra_sparse(G, source, target):
    visited = set()
    queue = [(0, source, None)]  # (tentative cost, vertex, parent)
    parents = {}
    
    low = 0
    while queue:
        vertex_cost, vertex, parent = queue[low]

        if vertex not in visited:
            visited.add(vertex) # Consider vertex as visited
            parents[vertex] = parent

            # Relax neighbor vertices of vertex
            for _, neighbor, edge_length in G.out_edges(vertex, data="length"):
                if neighbor not in visited:
                    neighbor_cost = vertex_cost + edge_length 
                    queue.append((neighbor_cost, neighbor, vertex))

        low += 1

    return parents